<img src="dh_logo.png" align="right" width="50%">

# Sistema de Recomendação de Filmes - Filtragem Colaborativa.

#### Agora vamos utilizar o conceito de filtragem colaborativa para fazer recomendações de filmes.

In [35]:
import pandas as pd

In [36]:
f = open('u.data').read()
print(f[0 : 20])
f[0: 20]

0	50	5	881250949
0	1


'0\t50\t5\t881250949\n0\t1'

In [37]:
cols = ['user_id', 
        'item_id', 
        'rating', 
        'timestamp'
       ]

int_matrix = pd.read_csv('u.data', 
                         sep = '\t', 
                         names = cols
                        )
int_matrix.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [38]:
movie_data = pd.read_csv('Movie_Id_Titles')
movie_data.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [39]:
int_matrix = int_matrix.merge(movie_data, 
                              on = 'item_id'
                             )
int_matrix.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [40]:
sparse_matrix = int_matrix.pivot_table(values = 'rating', 
                                       index = 'user_id', 
                                       columns = 'title'
                                      )
sparse_matrix.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
user_id = 4
user_best_movies = int_matrix.loc[int_matrix.user_id == user_id, : ].sort_values(by = 'rating', 
                                                                                 ascending = False
                                                                                ).head(10).title.tolist()
user_best_movies

['Star Wars (1977)',
 "Ulee's Gold (1997)",
 'Assignment, The (1997)',
 'Desperate Measures (1998)',
 'Incognito (1997)',
 'Air Force One (1997)',
 'Wedding Singer, The (1998)',
 'Lost Highway (1997)',
 'In & Out (1997)',
 'Cop Land (1997)']

In [42]:
user_best_movies[0]

'Star Wars (1977)'

In [43]:
sparse_matrix[user_best_movies[0]]

user_id
0      5.0
1      5.0
2      5.0
3      NaN
4      5.0
      ... 
939    NaN
940    4.0
941    NaN
942    5.0
943    4.0
Name: Star Wars (1977), Length: 944, dtype: float64

In [44]:
sparse_matrix.corrwith(sparse_matrix[user_best_movies[0]]).sort_values(ascending = False)

title
Hollow Reed (1996)                         1.0
Commandments (1997)                        1.0
Cosi (1996)                                1.0
No Escape (1994)                           1.0
Stripes (1981)                             1.0
                                          ... 
Wonderland (1997)                          NaN
Wooden Man's Bride, The (Wu Kui) (1994)    NaN
Yankee Zulu (1994)                         NaN
You So Crazy (1994)                        NaN
Á köldum klaka (Cold Fever) (1994)         NaN
Length: 1664, dtype: float64

In [45]:
int_matrix.groupby('title', 
                   sort = False
                  ).rating.count().head()

title
Star Wars (1977)                   584
Empire Strikes Back, The (1980)    368
Gone with the Wind (1939)          172
Kolya (1996)                       117
L.A. Confidential (1997)           297
Name: rating, dtype: int64

In [46]:
movie_data.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [47]:
movie_data = movie_data.merge(int_matrix.groupby('title').rating.count(),
                              left_on = 'title', 
                              right_index = True)
movie_data.rename({'rating':'num_rating'}, 
                  axis = 1, 
                  inplace = True
                 )
movie_data.head()

,item_id,title,num_rating
0,1,Toy Story (1995),452
1,2,GoldenEye (1995),131
2,3,Four Rooms (1995),90
3,4,Get Shorty (1995),209
4,5,Copycat (1995),86


In [48]:
thr = 50

In [49]:
user_id = 4
user_best_movies = int_matrix.loc[int_matrix.user_id == user_id,:].sort_values(by = 'rating', 
                                                                               ascending = False
                                                                              )
user_best_movies.head()

,user_id,item_id,rating,timestamp,title
80,4,50,5,892003526,Star Wars (1977)
37609,4,303,5,892002352,Ulee's Gold (1997)
92414,4,359,5,892002352,"Assignment, The (1997)"
89026,4,329,5,892002352,Desperate Measures (1998)
85079,4,361,5,892002353,Incognito (1997)


In [50]:
user_best_movies_with_ratings = user_best_movies.merge(movie_data.loc[ : ,['item_id','num_rating']], 
                                                       on = 'item_id')

user_best_movies_with_ratings = user_best_movies_with_ratings.loc[user_best_movies_with_ratings.num_rating > thr, :]
user_best_movies_with_ratings.head()

,user_id,item_id,rating,timestamp,title,num_rating
0,4,50,5,892003526,Star Wars (1977),584
1,4,303,5,892002352,Ulee's Gold (1997),184
3,4,329,5,892002352,Desperate Measures (1998),72
5,4,300,5,892001445,Air Force One (1997),431
6,4,354,5,892002353,"Wedding Singer, The (1998)",72


In [51]:
movies_with_low_ratings = movie_data.loc[movie_data.num_rating < thr, : ].title

In [52]:
sparse_matrix_min_ratings =  sparse_matrix.drop(movies_with_low_ratings, 
                                                axis = 1
                                               )
sparse_matrix_min_ratings.head()

title,101 Dalmatians (1996),12 Angry Men (1957),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),"39 Steps, The (1935)",Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Addams Family Values (1993),...,Willy Wonka and the Chocolate Factory (1971),Wings of Desire (1987),"Wings of the Dove, The (1997)",Winnie the Pooh and the Blustery Day (1968),"Wizard of Oz, The (1939)",Wolf (1994),"Wrong Trousers, The (1993)",Wyatt Earp (1994),Young Frankenstein (1974),Young Guns (1988)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,5.0,NaN,3.0,4.0,NaN,NaN,3.0,3.0,NaN,...,4.0,NaN,NaN,NaN,4.0,NaN,5.0,NaN,5.0,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
user_movies_to_cor = user_best_movies_with_ratings.title.head(10).tolist()
user_movies_to_cor

['Star Wars (1977)',
 "Ulee's Gold (1997)",
 'Desperate Measures (1998)',
 'Air Force One (1997)',
 'Wedding Singer, The (1998)',
 'Lost Highway (1997)',
 'In & Out (1997)',
 'Cop Land (1997)',
 'Contact (1997)',
 'Liar Liar (1997)']

In [54]:
sparse_matrix_min_ratings.corrwith(sparse_matrix_min_ratings[user_movies_to_cor[3]]).sort_values(ascending=False)

title
Air Force One (1997)                                        1.000000
Black Sheep (1996)                                          0.766259
Copycat (1995)                                              0.711744
Tales from the Crypt Presents: Bordello of Blood (1996)     0.639372
Addicted to Love (1997)                                     0.576379
                                                              ...   
Notorious (1946)                                           -0.331616
Nosferatu (Nosferatu, eine Symphonie des Grauens) (1922)   -0.338817
Third Man, The (1949)                                      -0.353730
Thin Man, The (1934)                                       -0.354700
Man Who Would Be King, The (1975)                          -0.459755
Length: 605, dtype: float64

In [55]:
movie_summary = pd.DataFrame()
for movie in user_movies_to_cor:
    movie_summary = pd.concat([movie_summary, sparse_matrix_min_ratings.corrwith(sparse_matrix_min_ratings[movie]
                                                                                ).sort_values(ascending = False).head(10)], 
                              axis = 0
                             )
    print(movie_summary.shape)
    
movie_summary.columns = ['correlation']
movie_summary

(10, 1)
(20, 1)
(30, 1)
(40, 1)
(50, 1)
(60, 1)
(70, 1)
(80, 1)
(90, 1)
(100, 1)


,correlation
Star Wars (1977),1.000000
"Empire Strikes Back, The (1980)",0.748353
Return of the Jedi (1983),0.672556
Raiders of the Lost Ark (1981),0.536117
Giant (1956),0.488093
...,...
Flubber (1997),0.494258
Dumb & Dumber (1994),0.494006
"Mask, The (1994)",0.484650
"Island of Dr. Moreau, The (1996)",0.477731


In [56]:
suggested_movies = movie_summary.loc[movie_summary.correlation < 0.9999, : ].sort_values(by = 'correlation', 
                                                                                         ascending = False
                                                                                        ).reset_index().drop_duplicates(subset = 'index'
                                                                                                                       ).head(10)
suggested_movies

,index,correlation
0,"Secret Garden, The (1993)",0.946729
1,"Thin Man, The (1934)",0.944911
2,"Amityville Horror, The (1979)",0.918559
3,"Santa Clause, The (1994)",0.905822
4,Antonia's Line (1995),0.895761
5,Muppet Treasure Island (1996),0.894792
6,Trees Lounge (1996),0.894714
7,Weekend at Bernie's (1989),0.892994
8,My Favorite Year (1982),0.890084
9,Breaking the Waves (1996),0.877058


In [57]:
def recommender(user_id, interation_file = 'u.data', movie_data_file = 'Movie_Id_Titles', thr = 50, 
                user_movies = 10, corr_movies = 10, suggestions = 10, cols = ['user_id', 'item_id', 
                                                                              'rating', 'timestamp'
                                                                             ]
               ):
    
    int_matrix = pd.read_csv(interation_file, sep = '\t', 
                             names = cols
                            )
    movie_data = pd.read_csv(movie_data_file)
    
    int_matrix = int_matrix.merge(movie_data.loc[ : , ['item_id','title']], 
                                  on = 'item_id'
                                 )
    movie_data = movie_data.merge(int_matrix.groupby('title').rating.count(), 
                              left_on = 'title', 
                              right_index = True
                                 )
    
    movie_data.rename({'rating':'num_rating'}, 
                      axis = 1, 
                      inplace = True
                     )
    
    movie_data = movie_data.loc[movie_data.num_rating > thr, :]
    int_matrix = int_matrix.loc[int_matrix.item_id.isin(movie_data.item_id), : ]
    
    sparse_matrix = int_matrix.pivot_table(values = 'rating', 
                                           index = 'user_id', 
                                           columns = 'title'
                                          )
    
    user_best_movies = int_matrix.loc[int_matrix.user_id == user_id, : ].sort_values(by = 'rating', 
                                                                                     ascending = False
                                                                                    ).head(user_movies).title.tolist()
    movie_summary = pd.DataFrame()

    for movie in user_best_movies:
        movie_summary = pd.concat([movie_summary, sparse_matrix.corrwith(sparse_matrix[movie]).sort_values(ascending = False).head(corr_movies)], 
                                  axis = 0
                                 )
        
    movie_summary.columns = ['correlation']
    suggested_movies = movie_summary.loc[movie_summary.correlation < 0.9999, : ].sort_values(by = 'correlation',
                                                                                             ascending = False
                                                                                            ).reset_index().drop_duplicates(subset = 'index'
                                                                                                                           ).head(10).loc[ : , 'index'].tolist()
    
    response = pd.concat([pd.Series(user_best_movies), pd.Series(suggested_movies)], 
                         axis = 1
                        )
    
    response.columns = ['watched', 'suggested']

    return response

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
item_based = recommender(20)
item_based

,watched,suggested
0,It's a Wonderful Life (1946),Crash (1996)
1,"Ghost and the Darkness, The (1996)",Roman Holiday (1953)
2,Searching for Bobby Fischer (1993),Hercules (1997)
3,Braveheart (1995),Breaking the Waves (1996)
4,Jurassic Park (1993),Desperate Measures (1998)
5,"Christmas Carol, A (1938)",Home for the Holidays (1995)
6,Jungle2Jungle (1997),Kolya (1996)
7,Beauty and the Beast (1991),Tin Men (1987)
8,Ransom (1996),Last Man Standing (1996)
9,"Lost World: Jurassic Park, The (1997)",Bed of Roses (1996)


# Próximos passos
1. Split de treino/teste
2. Recomendar com base nos usuários
3. Usar o método de clusterização no lugar do método de correlação
4. Reduzir dimensionalidade
5. O que fazer com os filmes com baixo número de avaliações
6. Ajustar avaliação de cada usuário pela sua avaliação média
7. Analisar os dados para entender como são as avaliações (ex.: média de avaliações está muito acima da média do intervalo possível)
8. Enriquecer os dados (descrições sobre os filmes)
9. Recomendação baseada em conteúdo

## Modelo de recomendação com base nos usuários

In [26]:
interation_file = 'u.data'
movie_data_file = 'Movie_Id_Titles'
thr_num_ratings_movie = 50
thr_num_ratings_user = 0

cols = ['user_id','item_id','rating','timestamp']
int_matrix = pd.read_csv(interation_file, 
                         sep = '\t', 
                         names = cols
                        )

movie_data = pd.read_csv(movie_data_file)

movie_data = movie_data.merge(int_matrix.groupby('item_id').rating.count(), 
                              left_on = 'item_id', 
                              right_index = True
                             )
user_data = int_matrix.groupby('user_id', 
                               as_index = False
                              ).rating.count()

user_data_min_ratings = user_data.loc[user_data.rating > thr_num_ratings_user, : ]

print(user_data.shape[0])
user_data_min_ratings.shape[0]

944


944

In [27]:
movie_data.rename({'rating':'num_ratings'}, 
                  axis = 1, 
                  inplace = True
                 )

movie_data = movie_data.loc[movie_data.num_ratings > thr, : ]

#int_matrix = int_matrix.loc[int_matrix.item_id.isin(movie_data.item_id), : ]
int_matrix = int_matrix.loc[int_matrix.user_id.isin(user_data_min_ratings.user_id), : ]

sparse_matrix = int_matrix.pivot_table(values = 'rating', 
                                       index = 'user_id', 
                                       columns = 'item_id'
                                      ).T

In [28]:
user = 20
items_users = int_matrix.loc[int_matrix.user_id == user, 
                             'item_id'].dropna()

list_of_users = int_matrix.user_id.unique().tolist()

thr_comparable = 20

comparable_users = []
for user_loop in list_of_users:
    if int_matrix.loc[int_matrix.user_id == user_loop, 'item_id'].dropna().isin(items_users).sum() > thr_comparable:
        comparable_users.append(user_loop)            

In [29]:
most_similar_user = sparse_matrix.loc[ : , comparable_users].corrwith(sparse_matrix.loc[ : , user]).sort_values(ascending = False).index[1]
most_similar_user

711

In [30]:
list_movies_most_similar_user = int_matrix.loc[int_matrix.user_id == most_similar_user, : ].dropna()
suggested_movies = list_movies_most_similar_user.loc[~list_movies_most_similar_user.item_id.isin(items_users)].sort_values(by = 'rating', 
                                                                                                                           ascending = False
                                                                                                                          ).head(10)
suggested_movies

,user_id,item_id,rating,timestamp
66155,711,213,5,879994390
54899,711,724,5,879995461
57390,711,8,5,879993707
73918,711,736,5,879993871
56573,711,277,5,879991476
56543,711,89,5,879993997
56404,711,966,5,879994390
75546,711,312,5,883589763
75825,711,427,5,886030557
76213,711,475,5,876185673


In [31]:
user_based = movie_data.loc[movie_data.item_id.isin(suggested_movies.item_id), 'title']
user_based

7                                         Babe (1995)
88                                Blade Runner (1982)
212                        Room with a View, A (1986)
276                                Restoration (1995)
311    Midnight in the Garden of Good and Evil (1997)
426                      To Kill a Mockingbird (1962)
474                              Trainspotting (1996)
723                          Circle of Friends (1995)
735                                Shadowlands (1993)
Name: title, dtype: object

In [32]:
pd.concat([item_based,user_based.reset_index()], 
          axis = 1, 
          ignore_index = True
         )

,0,1,2,3
0,It's a Wonderful Life (1946),Crash (1996),7.0,Babe (1995)
1,"Ghost and the Darkness, The (1996)",Roman Holiday (1953),88.0,Blade Runner (1982)
2,Searching for Bobby Fischer (1993),Hercules (1997),212.0,"Room with a View, A (1986)"
3,Braveheart (1995),Breaking the Waves (1996),276.0,Restoration (1995)
4,Jurassic Park (1993),Desperate Measures (1998),311.0,Midnight in the Garden of Good and Evil (1997)
5,"Christmas Carol, A (1938)",Home for the Holidays (1995),426.0,To Kill a Mockingbird (1962)
6,Jungle2Jungle (1997),Kolya (1996),474.0,Trainspotting (1996)
7,Beauty and the Beast (1991),Tin Men (1987),723.0,Circle of Friends (1995)
8,Ransom (1996),Last Man Standing (1996),735.0,Shadowlands (1993)
9,"Lost World: Jurassic Park, The (1997)",Bed of Roses (1996),NaN,NaN
